### Set working folder and import packages

In [17]:
### Edit the line below for the path to the "cims-models" folder ###
%cd -q "C:/CIMS/dev/cims-models"


import os
from os import path
from datetime import datetime
from IPython import get_ipython
import pandas as pd
import numpy as np

import CIMS

print("The current folder is", os.getcwd())

The current folder is C:\CIMS\dev\cims-models


### Adjust scenario parameters and optional update files

In [18]:
# Only uncommented regions below will be run in the simulation
region_list = [
    'CIMS',
    'Row',
    'CAN',
    'BC',
    'AB',
    'SK',
    'MB',
    'ON',
    'QC',
    'AT',
]

# Only uncommented years below will be run in the simulation
year_list = [
    ### Historical ###
    2000,
    2005,
    2010,
    2015,
    2020,
    ### Forecast ###
    # 2025,
    # 2030,
    # 2035,
    # 2040,
    # 2045,
    # 2050,
]

# Only uncommented sectors below will be run in the simulation
# Uncomment individual sectors to calibrate one at a time
# Must use Exogenous prices (and optional exogenous demand) file below when calibrating
sector_list = [
    'Coal Mining',
    'Natural Gas', # Must run all regions due to Natural Gas Market
    # 'Petroleum Crude',
    'Petroleum Refining',
    'Electricity',
    'Biodiesel',
    'Ethanol',
    'Hydrogen',
    'Mining',
    'Industrial Minerals',
    'Iron and Steel',
    'Metal Smelting',
    'Chemical Products',
    'Pulp and Paper',
    'Light Industrial',
    'Construction',
    'Residential',
    'Commercial',
    'Transportation Personal',
    'Transportation Freight',
    'Waste',
    'Agriculture',
    'Forestry',
]



### Base model and standard files below are required for the model to run
model_path = 'csv/model'
# Model files should be located at: cims-models/path/file/file_XX.csv

### Base model to start initialisation
base_model = 'CIMS_base'

### Default values and parameter list files
default_path = 'csv/defaults/defaults_Parameters.csv'
list_path = 'csv/defaults/defaults_Lists.csv'

update_files = {}


### Required sector files (included in csv/model/)
sector_req = [
    'fuels',
    'transmission',
    'coal mining',
    'natural gas',
    'petroleum crude',
    'petroleum refining',
    'electricity',
    'biodiesel',
    'ethanol',
    'hydrogen',
    'mining',
    'industrial minerals',
    'iron and steel',
    'metal smelting',
    'chemical products',
    'pulp and paper',
    'light industrial',
    'construction',
    'residential',
    'commercial',
    'transportation personal',
    'transportation freight',
    'waste',
    'agriculture',
    'forestry',
    ]
if sector_req:
    if model_path not in update_files:
        update_files[model_path] = []
    update_files[model_path].extend(sector_req)


### Required model files (included in csv/model/)
model_req = [
    'DCC', # declining capital cost
    'DIC', # declining intangible cost (neighbour effect)
    'FIC', # fixed intangible cost; primarily used for calibration
    'market share limits', # use of limits should be minimised
    ]
if model_req:
    if model_path not in update_files:
        update_files[model_path] = []
    update_files[model_path].extend(model_req)


### Optional model files (included in csv/model/)
model_optional = [
    'exogenous prices',  # needed for correct calibration of historical years
    'exogenous demand',  # use this file when calibrating endogenous supply sectors
    ]
if model_optional:
    if model_path not in update_files:
        update_files[model_path] = []
    update_files[model_path].extend(model_optional)


### Optional scenario update files

ref_path = 'csv/policies/reference'
ref_policies = [
### Economy
    'Ref_carbon tax',
    'Ref_OBPS',
### Coal Mining
### Natural Gas Production
### Petroleum Crude
### Mining
### Electricity
    'Ref_coal phase out',
    'Ref_nuclear ban',
    'Ref_nuclear decommission',
    'Ref_clean electricity',
    'Ref_CER',
### Biodiesel
### Ethanol
### Hydrogen
### Petroleum Refining
### Industrial Minerals
### Iron and Steel
### Metal Smelting
### Chemical Products
### Pulp and Paper
### Light Industrial
### Residential
    'Ref_incandescent phase out',
### Commercial
### Transportation Personal
    'Ref_LDV ZEV_Federal', #include before Prov version
    'Ref_LDV ZEV_Prov',
    'Ref_renewable fuel content_Fed', #include before Prov version
    'Ref_renewable fuel content_Prov',
### Transportation Freight
### Waste
    'Ref_waste methane large sites',
### Agriculture
    ]
if ref_policies:
    if ref_path not in update_files:
        update_files[ref_path] = []
    update_files[ref_path].extend(ref_policies)


# These files turn off existing Reference policies starting in the first model year
# Off files must be updated annually to reflect the last common year between a set of scenarios
turn_off_path = 'csv/policies/turn off'                                           
turn_off_policies = [
### Economy
    # 'Off_OBPS',
### Coal Mining
### Natural Gas Production
### Petroleum Crude
### Mining
### Electricity
### Biodiesel
### Ethanol
### Hydrogen
### Petroleum Refining
### Industrial Minerals
### Iron and Steel
### Metal Smelting
### Chemical Products
### Pulp and Paper
### Light Industrial
### Residential
### Commercial
### Transportation Personal
### Transportation Freight
### Waste
### Agriculture
    ]
if turn_off_policies:
    if turn_off_path not in update_files:
        update_files[turn_off_path] = []
    update_files[turn_off_path].extend(turn_off_policies)


scenario_path = 'csv/policies/net zero'    # Change this path as necessary based on current scenario
scenario_policies = [
### Economy
    # 'NZ_carbon tax',
### Coal Mining
### Natural Gas Production
    # 'NZ_natural gas production',
### Petroleum Crude
    # 'NZ_petroleum crude',
### Mining
### Electricity
    # 'NZ_electricity generation',
### Biodiesel
### Ethanol
### Hydrogen
### Petroleum Refining
### Industrial Minerals
### Iron and Steel
### Metal Smelting
### Chemical Products
### Pulp and Paper
### Light Industrial
### Residential
### Commercial
### Transportation Personal
### Transportation Freight
### Waste
### Agriculture
    ]
if scenario_policies:
    if scenario_path not in update_files:
        update_files[scenario_path] = []
    update_files[scenario_path].extend(scenario_policies)



### Scenario Name
### This will be the save location for results (i.e., results_dir/scenario_name/results_general.csv)
scenario_name = 'Reference'  # Set this to current scenario (e.g., "Reference", "Net Zero")


### Data frames, validate, graph

In [19]:
#################### Construct data frames ####################

# Need to reload functions in case changes were made to code
%reload_ext autoreload
%autoreload 2

time_start = datetime.now()

#################### Global parameters ###########################

# Leave the column list as-is to run the simulation
col_list1 = [
    'Branch',
    'Region',
    'Sector',
    'Technology',
    'Parameter',
    'Context',
    'Sub_Context',
    'Target',
    'Source',
    'Unit',
]
col_list2 = pd.Series(np.arange(2000,2051,5), dtype='string').tolist()
col_list = col_list1 + col_list2


#################### Build Model ###########################

print(f'Loading base model: {base_model}')
load_paths = []
for reg in region_list:
    cur_path = f'{model_path}/{base_model}/{base_model}_{reg}.csv'
    if os.path.exists(cur_path):
        print(f'\t{reg} - loaded')
        load_paths.append(cur_path)
    else:
        print(f'\t{reg} - file does not exist')

print(f'\nLoading model updates files:')
if bool(update_files):
    update_paths = []
    for dir, file in update_files.items():
        for file in update_files[dir]:
            print(f'\n{file}')
            for reg in region_list:
                cur_path = f'{dir}/{file}/{file}_{reg}.csv'
                if os.path.exists(cur_path):
                    print(f'\t{reg} - loaded')
                    update_paths.append(cur_path)
                else:
                    print(f'\t{reg} - file does not exist')
else:
    print('None')

model_validator = CIMS.ModelValidator(
    csv_file_paths = load_paths,
    csv_update_file_paths = update_paths,
    col_list = col_list,
    year_list = year_list,
    sector_list = sector_list,
    default_values_csv_path = default_path,
    list_csv_path = list_path,
)

print()
print('File reading and model loading: ', datetime.now() - time_start)

Loading base model: CIMS_base
	CIMS - loaded
	Row - loaded
	CAN - loaded
	BC - loaded
	AB - loaded
	SK - loaded
	MB - loaded
	ON - loaded
	QC - loaded
	AT - loaded

Loading model updates files:

fuels
	CIMS - loaded
	Row - file does not exist
	CAN - file does not exist
	BC - loaded
	AB - loaded
	SK - loaded
	MB - loaded
	ON - loaded
	QC - loaded
	AT - loaded

transmission
	CIMS - loaded
	Row - file does not exist
	CAN - file does not exist
	BC - file does not exist
	AB - file does not exist
	SK - file does not exist
	MB - file does not exist
	ON - file does not exist
	QC - file does not exist
	AT - file does not exist

coal mining
	CIMS - file does not exist
	Row - file does not exist
	CAN - file does not exist
	BC - loaded
	AB - loaded
	SK - loaded
	MB - file does not exist
	ON - file does not exist
	QC - file does not exist
	AT - loaded

natural gas
	CIMS - file does not exist
	Row - loaded
	CAN - loaded
	BC - loaded
	AB - loaded
	SK - loaded
	MB - loaded
	ON - loaded
	QC - loaded
	A

In [20]:
#################### Validate scenario ####################
time_start = datetime.now()

model_validator.validate(verbose=False)

print()
print('Model validation: ', datetime.now() - time_start)


*** Errors ***
No errors found!

*** Warnings ***
47 nodes are defined in the model, but are not requested by other nodes. See ModelValidator.warnings['unrequested_nodes'] for more info.
219 nodes or technologies request no services. See ModelValidator.warnings['nodes_no_requested_service'] for more info.
7 nodes/technologies request the same service more than once. See ModelValidator.warnings['duplicate_service_requests'] for more info.
293 nodes are defined in the model, but are only requested by nodes where all 'Service requested' values are 0. See ModelValidator.warnings['zero_requested_nodes'] for more info.

Model validation:  0:00:07.079450


In [21]:
#################### Show validator warnings ####################
# change warning type as needed to view indicated nodes/techs
model_validator.warnings['inconsistent_tech_refs']

[]

In [22]:
#################### Construct graph ####################

model = CIMS.Model(
    csv_init_file_paths = load_paths,
    csv_update_file_paths = update_paths,
    col_list = col_list,
    year_list = year_list,
    sector_list = sector_list,
    default_values_csv_path = default_path,
    list_csv_path = list_path,
)

print()
print('File reading and model loading: ', datetime.now() - time_start)

KeyboardInterrupt: 

### Run scenario and output results

In [ ]:
#################### Run scenario and output results ###########################
time_start = datetime.now()

model.run(equilibrium_threshold=0.05, max_iterations=10, show_warnings=False, print_eq=True)

results_path = f'results/{scenario_name}'
isExist = os.path.exists(results_path)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(results_path)

CIMS.log_model(model=model,
                output_file = f'{results_path}/results_general.csv',
                path = 'results/results_general.txt')

print()
print('Simulation time: ', datetime.now() - time_start)

print()
print(f"Results exported to '{scenario_name}'")

In [ ]:
#################### Export tech results ####################
import os

results_path = f'results/{scenario_name}'
isExist = os.path.exists(results_path)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(results_path)

CIMS.log_model(model=model,
                output_file = f'{results_path}/results_tech.csv',
                path = 'results/results_tech.txt')
print('\n')
print('Results exported')

In [ ]:
#################### Print service request loops after running model ####################

import pprint
pprint.pp(model.loops)